<h1 style="text-align: center;"> A* Algorithm for Multi-Address Drone Delivery in a Grid-Based City Map</h1>



***
Course: COSC2968/COSC3053\
Semester: 2024-2\
Assignment: 2 - OPTION A (PROGRAMMING) Classical AI\
Author: Hoang Minh Thang\
Student ID: s3999925\
Compiler used: Python 3.12.4\
Due date: 14/08/2024 at 17:00
***

### a. Introduction

Suppose you are employed by a delivery company that uses drones to deliver packages. Your job is to create an AI program that uses the A* (A star) algorithm to determine the optimal flight paths for delivery drones.
Throughout a metropolis, the drone must deliver packages to several addresses. The city is shown as a grid map, with each cell denoting an open or restricted area. Obstacles such as towering buildings, trees, and airports are known as blocked cells, and they prevent drones from entering them. [[<sup id="fn1">1</sup>](#fn1)]

### b. User Instructions

**In order to run the program successfully, user have to do following steps:**

>1. Press the $\boxed{Run All}$ button at the top left, next to the $\boxed{Markdown}$ button.
>2. The program will then prompt you to enter the number corresponding to the city map you want to deliver [1 for Nevada state, 2 for New York City, 3 for Chicago City, 4 for Shanghai City]. 
>3. Please enter the number [1, 2, 3 or 4] and press enter.
>4. Don't forget to close the visualize window and press the $\boxed{Clear All Outputs}$ button at the top left next to the $\boxed{Restart}$ button *before second execution.*

### c. Note
**The provided code includes the following notes:**

> 1. In this program, the location point (address) is represented by coordinates (x, y). Where:
>$${->\text{x is the distance of this point along the \underline{row-axis} from the original.}}$$
>$${->\text{y is the distance of this point along the \underline{column-axis} from the original.}}$$
> 2. This visualization map does not have its legend display next to since the Tkinter library does not provide any related built-in functions. Instead, **the legend will be represented in section IV**. 
> 3. This program can not execute visualization on the Jupyter Notebook localhost since it uses the Tkinter library. **It is highly recommended to run this file in VSCode with all Jupyter extensions installed.** 
> 4. The analysis of the solution's efficiency will be displayed in the corresponding console.

*The program requires the following libraries:*

In [ ]:
import tkinter as tk # used for visualization a city (2D map)
import time # used for calculate total time it takes to compute the final path.
import sys # used for measure memory usage of the path and a grid (2d map)

## I. Heuristic Function Method Justification 
#### 1. Overview Heuristic Function
There are 3 well-known heuristic functions for 2D grid maps: [[<sup id="fn1">2</sup>](#fn1)]
- Manhattan distance ($L_1$)
- Euclidean distance ($L_2$)
- Diagonal distance ($L_∞$) (Chebyshev distance & Octile distance)

>According to the given assessment task, the $\textbf{Chebyshev distance}$ is the most appropriate heuristic for this implementation. 

#### 2. Justification
My chosen metric is **Chebyshev distance** because it is suitable for use as a heuristic in 2D grid-based pathfinding, especially when the drone can move diagonally. Because it takes into account the maximum distance in any direction (horizontally or vertically, or even diagonally), the Chebyshev distance is a good estimate of the grid’s two points apart.[[<sup id="fn1">2</sup>](#fn1)]

In a grid based system, *the Chebyshev distance is better than Euclidean distance, especially when there can be diagonal movements by drones*. This is attributed to the fact that Euclidean measures straight line distances between two points whilst Chebyshev considers distances in all directions.

___Benefits:___
- *Efficiency*: Calculating the Chebyshev distance is computationally efficient and minimal calculations for real-time decisions.
-	*Admissibility*: The Chebyshev distance is never overestimates the true distance to the goal, which is important for the A* algorithm to work correctly.
- *Scalability*: This heuristic scales well with the size of the city and the number of delivery zones. It remains effective regardless of the complexity of the city’s layout.

___Limitations:___
-	The Chebyshev distance is *not perfect and not suitable for 3D grid map. For example if 3D grid has many obstacles or narrow passages, Chebyshev distance may not give the shortest path.
-	The Chebyshev distance does not take into account the drone's movement constraints, such as its speed and acceleration.
-   The length of the path may not be optimal if the current start point is too far from the delivery point.

_Therefore, I went for the **Chebyshev distance heuristic** because it is an appropriate heuristic for grid-based pathfinding, particularly when drones can move diagonally. It results in eight directions (up, down, left, right, and diagonals) [[<sup id="fn1">4</sup>](#fn1)]. It gives a good approximation of the distance between two points in a **2D grid map** that is needed by the A* algorithm to find the shortest path. Additionally, the total path length computed by the Chebyshev distance is much less than that compared to Manhattan distance and Euclidean distance._


#### 3. Overview Chebyshev distance
The Chebyshev distance, also known as the L∞ distance, is distance metric accounts for diagonal movement and is defined as the maximum of the absolute differences between the coordinates of two points.[[<sup id="fn1">3</sup>](#fn1)], which is illustrated in the formula below:
> - If we have two points($x_1,y_1$) and ($x_2, y_2$) then:
>    $$\boxed {D_\text{Chebyshev} = max(|x_2 - x_1|, |y_2 - y_1|)}$$ 

*Here is an implementation of the Chebyshev distance heuristic:*

In [ ]:
# Calculate the Chebyshev distance between two locations.
def chebyshev_distance(node, goal):
    """
    Calculate the Chebyshev distance between two points in a grid.

    Args:
        node (tuple): (x, y) coordinates of the current node
        goal (tuple): (x, y) coordinates of the next node

    Returns:
        int: Chebyshev distance between the two points (locations)
    """
    dx = abs(goal[0] - node[0])
    dy = abs(goal[1] - node[1])
    return max(dx, dy)

## II. A* Implementation
### 1. A* algorithm applies the Chebyshev distance heuristic
Here's the implementation of the A* search algorithm [<sup id="fn1">4</sup>](#fn1) between 2 locations:

In [ ]:
# Perform an A* search to find the optimal path between 2 locations
def searchPath(city_map, start_location, delivery_location):
    """
    Arguments:
        city_map (list): The city map.
        start_location (tuple): The starting location of the drone.
        delivery_location (tuple): The delivery location.
    
    Returns:
        list: The shortest path from start location to the delivery location.
    """
    # Create a priority queue to store the nodes to visit
    queue = []
    
    # Create a dictionary to store the cost to reach each node
    cost_to_reach = {start_location: 0}
    
    # Create a dictionary to store the previous node in the path
    previous_node = {}
    
    # Add the starting node to the queue
    queue.append((start_location, 0))
    
    while queue:
        # Get the node with the lowest cost from the queue
        current_node, current_cost = queue.pop(0)
        
        # Check if we've reached the delivery location
        if current_node == delivery_location:
            
            # Reconstruct the path to the delivery location
            path = []
            while current_node in previous_node:
                path.append(current_node)
                current_node = previous_node[current_node]
            path.append(start_location)
            path.reverse()
            return path
        
        # Explore the neighbors of the current node
        for dx, dy in [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]: # Adjacent squares
            x, y = (current_node[0] + dx, current_node[1] + dy)
            
            # Check if the neighbor is within the city map and is not an obstacle
            if 0 <= x < len(city_map) and 0 <= y < len(city_map[0]) and city_map[x][y] == 0:
                
                # Calculate the cost to reach the neighbor
                if (abs(dx) + abs(dy)) == 1:  # Horizontal or vertical movement
                    new_cost = current_cost + 1
                else:  # Diagonal movement
                    new_cost = current_cost + 1.4  # Increase the cost for diagonal movement
                
                # Check if we've already visited the neighbor with a lower cost
                if (x, y) not in cost_to_reach or new_cost < cost_to_reach[(x, y)]:
                    # Update the cost to reach the neighbor
                    cost_to_reach[(x, y)] = new_cost
                    
                    # Calculate the heuristic cost to reach the delivery location
                    heuristic_cost = chebyshev_distance((x, y), delivery_location)
                    
                    # Add the neighbor to the queue
                    queue.append(((x, y), new_cost + heuristic_cost))
                    
                    # Update the previous node in the path
                    previous_node[(x, y)] = current_node
    
    # Drone failed to find a path to the delivery location
    return None


### 2. Multiply addresses (goals) 
The function below is used to find the best optimal path between the starting position and all the multiple addresses needed for delivery in the corresponding city that the user has chosen.

In [ ]:
def optimal_path_solution(city_map, start_location, address_point):
    """
    Arguments:
        city_map (list): The city map.
        start_location (tuple): The starting location of the drone.
        address_point (list): The collection of multiple delivery location.
    
    Returns:
        list: The optimal path from start location to the last delivery address location.
    """
    # Declare variable
    optimal_path = [] 
    final_path = []

    for point in range(len(address_point)):
                path = searchPath(city_map, start_location, address_point[point]) # Call the a_star_search function\
                
                if (path == None): # Not found path then set 'optimal_path' variable to None and return 
                        optimal_path = None
                        return None 
                
                start_location = path[-1] # Update current position
                for index in range(len(path)):
                        optimal_path.append(path[index]) # Update the final optimal path
          
# Remove unnecessary tuples from a optimal_path

    # final_path = [point for index, point in enumerate(optimal_path) if index == 0 or point != optimal_path[index-1]]

    # Iterate over each tuple in the optimal_path list with enumerate
    for index, point in enumerate(optimal_path):
       # Check if it's the first element or if it's different from the previous one
        if index == 0 or point != optimal_path[index - 1]:
            # Remove the tuple to the final_path list
            final_path.append(point)

    return final_path

## III. Support function
### 1. Sort the delivery points list
 This function below ensures that the drone delivers packages to the addresses in the most efficient order, starting from the closest address and moving towards the farthest address.

 Note:
 - This function is not necessary to be called in this program since it may affect the optimal path and make it suboptimal instead.

In [ ]:
def sort_address_points(start_point, address_points, heuristic_function):
        '''
        Arguments:
            start_point (tuple): The starting location of the drone.
            address_points (list): The collection of multiple delivery location.
        Returns:
            list: The sorted list of delivery locations.
        '''
        # Sort the list of multiple adddress in ascending order.
        address_sorted = [(end, heuristic_function(start_point, end)) for end in address_points]
        address_sorted.sort(key = lambda x: x[1])
        return [point[0] for point in address_sorted] 

### 2. Modify the map 

The function 'modify_map' below is used to prepare the city map for the drone's delivery journey, by marking the starting location and the delivery locations on the map. The modified map is then used as input for visualize that map and optimal path for the drone to deliver to all the specified addresses.

In [ ]:
def modify_map(city_maze,start_point, end_point):
    """
    Arguments:
        city_maze (list): The city map.
        start_point (tuple): The starting location of the drone.
        end_point (list): The collection of multiple delivery location.
    Returns:
        list : Update city map (consist of starting location and multiple delivery location).
    """
    # Mark the start point on the map
    city_maze[start_point[0]][start_point[1]] = 2
    
    # Mark all end point on the map
    for point in end_point:
        city_maze[point[0]][point[1]] = 3
    
    return city_maze


### 3. Path length function

The function 'calculate_path_length' below calculates the total length (in units) of the optimal path found by the drone in the city map.
-   1 node horizontal distance or 1 node vertical distance = 1 unit.
-   1 node diagonal distance = $\sqrt{2}$ units 


In [ ]:
def calculate_path_length(path):
    '''
    Arguments:
        path (list): The optimal path of the drone for city_map.
    Returns:
        float: The total length of the optimal path in the city map.
    '''
    length = 0
    for i in range(1, len(path)):
        length += ((path[i][0] - path[i-1][0]) ** 2 + (path[i][1] - path[i-1][1]) ** 2) ** 0.5
    return length


## IV. Visualizes the final optimal delivery path on the city map using *tkinter library*

**Legend for the visualization map**

| $$Symbol$$                                                                  | $$Description$$                          |
|-----------------------------------------------------------------------------|------------------------------------------|
|<span style="color:rgb(238,201,0)">&#9724;</span> 'Gold' Square              | Empty space                              |
|<span style="color:rgb(85,107,47)">&#9724;</span> 'Dark-Olivie-Green' Square | Obstacle (eg. tree, building, etc.)      |
|<span style="color:rgb(205,112,84)">&#9724;</span> Salmon Square             | Starting point of the drone              |
|<span style="color:rgb(128,0,0)">&#9724;</span> Marron Square                | Delivery point (address)                 |
|<span style="color:rgb(46,139,87)">&#8594;</span> 'Sea Green' Line           | The final optimal path (the zone's path) |


*The "visualization" function bellow is responsible for creating a visual representation of the optimal path found by the drone on the city map using the Tkinter library.*

In [ ]:
def visualization(city_map, path, city_name):
    """
    Arguments:
        city_map (list): The city map.
        path (list): The optimal path of the drone for city_map.
        city_name (string): The name of the city (map)
    Returns:
        Visualize the optimal path on the city map 
    """
   
    # Create the Tkinter window
    root = tk.Tk()
    root.title("Map Visualization for " + city_name)

    # Calculate the canvas size based on the city map size
    cell_size = 30
    canvas_width = len(city_map[0]) * cell_size
    canvas_height = len(city_map) * cell_size

    # Create the canvas
    canvas = tk.Canvas(root, width = canvas_width, height = canvas_height)
    canvas.pack()
    
    # Draw the map (city)
    for y, row in enumerate(city_map):
        for x, cell in enumerate(row):
            if cell == 0: 
                color = "#EEC900" # Display the empty space as gold2 color on the map
            elif cell == 1:
                color = "#556B2F" # Display the obstacle as darkolivegreen color on the map
            elif cell == 2:
                color = "#CD7054" # Display the start location as salmon3 color on the map
            elif cell == 3:
                color = "#800000" # Display the delivery address as marron color on the map
            canvas.create_rectangle(x*cell_size, y*cell_size, (x+1)*cell_size, (y+1)*cell_size, fill = color) 

     # Create a frame to hold the path and evaluation labels
    frame = tk.Frame(root)
    frame.pack()

    # Ensure optimal path is exist
    if path is not None and len(path) > 0:
        
        # Draw the optimal path as the seagreen4 color line on the map
        for i in range(len(path)-1):
          canvas.create_line(path[i][1]*cell_size+(cell_size/2), path[i][0]*cell_size+(cell_size/2), path[i+1][1]*cell_size+(cell_size/2), path[i+1][0]*cell_size+(cell_size/2), fill = "#2E8B57", width = 5)
        
        # Splt the path into two halves
        first_half = path[:len(path)//2]
        second_half = path[len(path)//2:]
        
        # Display the optimal path on the canvas
        first_path_text = " -> ".join(map(str, first_half))
        second_path_text = " -> ".join(map(str, second_half))
        path_label = tk.Label(frame, text = f"Conclude: The drone successfully delivered the packages.\n\nPath: {first_path_text}\n{second_path_text}", font=("Helvetica", 12))
        path_label.pack()

    else:
        evaluation_text = "Conclude: Delivery failure (No path found)"
        evaluation_label = tk.Label(frame, text=evaluation_text, font=("Helvetica", 12))
        evaluation_label.pack()
        
    # Start the Tkinter event loop
    root.mainloop()


## V. Testing (main function)

To represent the city map, I will use a 2D array where each element represents a cell [0 for empty space and 1 for obstacles].\
I will also store the starting location of the drone and the list of addresses that drone need *to be deliver for each city (maze)*.

In [ ]:
def main():

        # Prompt user's choice for the map
        city = input("Which map will you select for delivery journey [1 for Nevada state, " + "2 for New York city, " + "3 for Chicago city, " + "4 for Shanghai city]")
        
        # Ensure user's choice for the map is correct 
        while city not in ['1', '2', '3', '4']:
                city = input("Invalid input. Please choose the valid city map (1, 2, 3 or 4): ")

        if city == '1': # (Case 1, No path found)
                # Define the map 1 (Nevada state)  (0 = empty, 1 = obstacle)    
                map =  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                        [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0],
                        [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0],
                        [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0],
                        [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
                        [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                        [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
                        [0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0],
                        [1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0],
                        [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
                        [0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0],
                        [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
                        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
                        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1],
                        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0] ]

                start_location = (0, 0) # Define the starting location of the drone
                delivery_location = [(9, 8), (6, 16), (19, 19)] # Define the list of delivery locations
                map_name = "Nevada state"
        elif city == '2': # (Case 2, Path found)
        # Define the map 2 (New York City) ->  (0 = empty, 1 = obstacle)
                map =  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
                        [0, 1, 0, 0, 0, 1, 1, 1, 1, 0],
                        [0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
                        [0, 1, 1, 1, 1, 1, 0, 0, 0, 0],
                        [0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                        [0, 0, 0, 1, 0, 0, 1, 0, 0, 0]]
                
                start_location = (3, 4) # Define the starting location of the drone
                delivery_location = [(0, 3), (1, 2), (7, 4), (9, 2)]  # Define the list of delivery locations
                map_name = "New York city"

        elif city == '3': #  (Case 3, Path found)
                # Define the map 3 (Chicago city) (0 = empty, 1 = obstacle)
                map =  [[0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                        [0, 0, 1, 1, 0, 1, 0, 0, 0, 0],
                        [0, 1, 0, 0, 0, 1, 1, 1, 1, 0],
                        [0, 1, 0, 0, 1, 1, 0, 0, 0, 0],
                        [0, 1, 1, 1, 1, 0, 0, 1, 0, 0],
                        [0, 0, 0, 1, 0, 0, 0, 0, 1, 1],
                        [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                        [0, 1, 0, 1, 0, 0, 0, 1, 0, 1],
                        [0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
                        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]
                
                start_location = (2, 4) # Define the starting location of the drone
                delivery_location = [(0, 2), (8, 2), (8, 9)]  # Define the list of delivery locations
                map_name = "Chicago city"

        elif city == '4': # (Case 4, Path found)
                # Define the map 4 (Shanghai city) (0 = empty, 1 = obstacle)
                map =  [[0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
                        [0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0],
                        [1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0],
                        [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0],
                        [0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0],
                        [0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0],
                        [0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0],
                        [0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0],
                        [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
                        [0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0],
                        [1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0],
                        [0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0],
                        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                        [1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0],
                        [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0],
                        [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0],
                        [0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0],
                        [0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0],
                        [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],]
                
                start_location = (0, 0) # Define the starting location of the drone
                delivery_location = [(5, 1), (10, 19), (15, 15), (19, 19)]  # Define the list of delivery locations
                map_name = "Shanghai city"
      
        
        #delivery_location = sort_address_points(start_location, delivery_location, heuristic_function=chebyshev_distance)

        print ("Case:", city)
        print ("City:", map_name)
        print ("Starting location:", start_location)
        print ("Delivery locations:", delivery_location)  
        # Testing with 3 different mazes

        start_time = time.time() # Record the start time
        optimal_path = optimal_path_solution(map, start_location, delivery_location) # Call the 'optimal_path_solution' function
        time.sleep(1)
        end_time = time.time() # Record the end time

        # Calculate the execution time
        execution_time = (end_time - start_time - 1)

        map_update = modify_map(map, start_location, delivery_location) # Call the 'modify_map' function
        if optimal_path:
                path_length = calculate_path_length(optimal_path) # Call the 'calculate_path_length' function

                print(f"Heuristic Choice: Chebyshev distance")
                print(f"The path taken by the A* algorithm is as follows:\n {optimal_path}\n")
                print(f"====> PACKAGES DELIVERY SUCCESSFULLY <====\n")
                print(f" Metric                               |  Value") 
                print(f"======================================|======================")
                print(f"Number of Nodes (include start point) |{len(optimal_path)}")  # Display the number of nodes
                print(f"Total Path cost (number of edges)     |{len(optimal_path)-1}")
                print(f"Execution time                        |{execution_time:.6f} seconds") # Display the execution time 
                print(f"Memory used                           |{sys.getsizeof(optimal_path) + sys.getsizeof(map)} bytes") # Display the memory used for the map and the path  
                print(f"Computed Path Length                  |{path_length:.3f} units")    # Display the optimal path length
        else: 
                print(f"\n===> DELIVERY FAILURE (no path found) <===")
       
        visualization(map_update, optimal_path, map_name) # Call the 'visualization' function

        return 

## VI. Analysis & Conclusion
##### Case 1: Nevada State
As shown in the city map visualization, the drone needs to deliver packages to two address points. Both of these points are surrounded by obstacles, which block all possible paths. As a result, the drone is unable to reach the destination points, making successful package delivery impossible in this case.
>>> Overall, the A* algorithm not successfully delivered packages in this scenarios.

##### Case 2: New York City
**Path Description:**
The A* algorithm was able to produce probable routes from the commencement point, Node 3, 4, to various delivery locations, Node 0, 3, Node 1, 2, Node 7, 4, Node 9, 2 in New York City. The compute path is also shown in the city map and connects such points. Even more importantly, the line is not straight, which means that the algorithm considered other intermediate points before arriving at the shortest distance.

**Technical Metrics:**
- Number of Nodes: In total, it encompasses 15 nodes, and one of them is the start node. Nodes represent cells of the grid.
- Total Path Cost: 14 , the physical distance that was taken by the algorithm was at its minimum. This efficiency is the one that is most crucial for the delivery of packages.
- Execution Time and Memory Usage: Notably, impressively about the A* algorithm the solution was reached with about 0.001289 seconds to make and it used up 320 bytes of memory space (city map and final path).

**Effectiveness Assessment:** 
Considering the constraints of an urban environment, where direct routes may be obstructed by unseen obstacles, this path is effective. The computed path length (approximately 17.314 units) suggests efficiency. 
>>> Hence, by apply the A* algorithm wth Chebyshev heuristic, the drone successfully delivered the packages.

##### Case 3: Chicago City
**Path Description:** 
The A* algorithm was able to identify potential routes starting from the location of the delivery hub (Node 2, 4) to several delivery locations (Node 0, 2, Node 8, 2, Node 8, 9,) in Chicago City. The drone's movement is also illustrated in the city map and it connects such points. Most important, the line is jagged meaning that the algorithm analyzed different mid-points to come up with the shortest distance.

**Technical Metrics:**
- Number of Nodes: The path comprises 19 nodes, including the start point. Each node represents a grid cell.
- Total Path Cost: 18, the algorithm minimized the actual distance traveled. This efficiency is crucial for package delivery.
- Execution Time and Memory Usage: Impressively, the A* algorithm executed approximately 0.001565 seconds and used up 384 bytes of memory  (city map and the final path).

**Effectiveness Assessment:** 
Considering the constraints of an urban environment, where direct routes may be obstructed by unknown obstacles, this path is effective. The computed path length (approximately 20.899 units) suggests efficiency. 
>>> In summary, the A* algorithm performed well, considering the provided metrics with its value and the Chebychev distance heuristic. The packages were delivered successfully along this path.

##### Case 4: Shanghai City
**Path Description:** 
The A* algorithm successfully navigated from the starting location (0, 0) to the specified delivery addresses (5, 1), (10, 19), (15, 15) and (19, 19) in Shanghai city. The delivery path, visually represented on the city map, connects those points. Notably, the path is non-linear, indicating that the algorithm considered various intermediate nodes to optimize the route.

**Technical Metrics:**
- Number of Nodes: The path comprises 38 nodes, including the start point. Each node represents a grid cell.
- Total Path Cost: 37, the algorithm minimized the actual distance traveled. This efficiency is critical for package delivery.
- Execution Time and Memory Usage: Impressively, the A* algorithm executed approximately 0.001744 seconds and used 592 bytes of memory (city map and the final path).

**Effectiveness Assessment:**
 Considering the constraints of an urban environment, where direct routes may be obstructed by unseen obstacles, this path is effective. The computed path length (approximately 41.556 units) suggests efficiency. 
>>> Therefore, by approach A* algorithm wth Chebyshev heuristic, the drone successfully delivered the packages.

##### NOTE

> **ALL TECHNICAL METRICS OF THE COMPUTED PATH (OPTIMAL PATH) ON THE CITY MAP ARE ILLUSTRATED BELOW:**

In [ ]:
if __name__ == '__main__':
    # Call the main function to execute the program
    main()

### References
[[<sup id="fn1">1</sup>](#fn1)] “Assignment 2 - OPTION A (PROGRAMMING): Classical AI.” Accessed: Aug. 10, 2024. [Online]. Available: https://rmit.instructure.com/courses/138595/assignments/969572<br>
[[<sup id="fn1">2</sup>](#fn1)] “Heuristics.” Accessed: Aug. 09, 2024. [Online]. Available: https://theory.stanford.edu/~amitp/GameProgramming/Heuristics.html<br>
[[<sup id="fn1">3</sup>](#fn1)] “(PDF) On A* Graph Search Algorithm Heuristics Implementation Towards Efficient Path Planning in the Presence of Obstacles.” Accessed: Aug. 09, 2024. [Online]. Available: https://www.researchgate.net/publication/368248729_On_A_Graph_Search_Algorithm_Heuristics_Implementation_Towards_Efficient_Path_Planning_in_the_Presence_of_Obstacles<br>
[[<sup id="fn1">4</sup>](#fn1)] “A* Search Algorithm,” GeeksforGeeks. Accessed: Aug. 10, 2024. [Online]. Available: https://www.geeksforgeeks.org/a-search-algorithm/

---